# Actor Critic pour résoudre l'environnement OpenAI Gym Cartpole

Copie expliquée de la page https://keras.io/examples/rl/actor_critic_cartpole/

## Introduction 

L'algorithme Actor Critic est une famille d'algorithme de Reinforcement Learning qui divise la tâche de sélection de l'action en deux parties à savoir :
- une partie (Critic) qui évalue l'intérêt ou la valeur de l'action au regard de l'environnement et du système considéré
- une partie (Actor) qui en utilisant les informations générées par le Critic choisit effectivement l'action pour l'état présent



***
On commence par importer les packages nécessaires au programme et on met en place l'environnement qui sera utilisé par l'algorithme. On définit ensuite quelques variables qui seront utiles au problème. 

In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keyboard

# Configuration parameters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
max_steps_per_episode = 10000
env = gym.make("CartPole-v0")  # Create the environment
env.seed(seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

***
On définit les deux Neural Networks qui représentent l'acteur et le critique. Le critique prend en entrée le vecteur issu de l'observation de l'état de l'environnement et renvoie en sortie la valeur calculée de l'état actuel. L'acteur prend également en entrée le vecteur observation mais en sortie renvoie une probabilité d'action pour chaque action possible. On verra par la suite que l'acteur se nourrit de la valeur estimée par le critique pour générer la distribution probabilistique d'exécuter les actions. En général, comme c'est donc fait dans l'implémentation suivante, le critique et l'acteur partage quelques-unes des premières couches de neurones pour accélérer la convergence du modèle. 

In [ ]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation = 'relu')(inputs)
action = layers.Dense(num_actions, activation = "softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs = inputs, outputs = [action, critic])

***
L'environnement est dit résolu lorsque la récompense totale moyenne atteint 195 pour 100 essais consécutifs. Aussi, on va effectuer un certains nombres d'itérations du programme non défini tant que la condition n'est pas atteinte. 

On commence donc par initialiser l'environnement avec la fonction habituelle `env.reset()` et créer des listes vides qui vont nous permettre de stocker les valeurs successives de variables. La première étape consiste à utiliser les réseaux acteur et critique pour déterminer une distribution probabilistique sur les actions possibles. Puis on choisit aléatoirement une action dans cette distribution que l'on applique à l'environnement en utilisant la fonction `env.step()`. Les éléments renvoyés sont stockés et on peut vérifier si l'exécution de l'environnement est terminé avec la variable `done`. 

A l'extérieur de l'exécution de l'épisode, on cumule la récompense avec celle des autres épisodes puis on calcule le retour attendu durant l'épisode qui vient d'être calculé. Le calcul du retour s'effectue avec la séquence de récompense dans l'ordre [inverse](https://stackoverflow.com/questions/31633635/what-is-the-meaning-of-inta-1-in-python). Le retour est alors normalisé et reformaté. Les pertes attribuées à l'acteur et au critique sont alors calculées durant tout l'épisode qui vient d'être accompli et leur somme globale est utilisée pour effectuer une descente de gradient sur les deux réseaux de neurones. 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history=[]
critic_value_history=[]
reward_history=[]
running_reward=0
episode_count=0

while True:
    state = env.reset()
    episode_reward=0
    with tf.GradientTape() as tape: # A retravailler
        for timestep in range(1, max_steps_per_episode):
            # env.render() # Si on veux le rendu
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)
            
            # pour l'état présent, renvoie les sorties des deux 2N Actor et Critic
            # stocke l'avantage pour des calculs ultérieurs hors de la boucle
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0,0])
            # print(action_probs)
            # print(critic_value)
            # Policy : donne l'action selon les actions possibles et la distribution d'actions donnée par l'actor
            # puis stocke cette action dans l'historique
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0,action]))
            
            # Joue l'environnement pour l'état présent et l'action précédemment choisie
            # puis stocke la récompense dans l'historique
            # Si done, alors le pendule est tombé et la simulation n'est plus intéressante.
            state, reward, done, _ = env.step(action)
            reward_history.append(reward)
            episode_reward += reward
            
            if done:
                break 
        
        running_reward = 0.05*episode_reward + (1-0.05)*running_reward
        
        # Calcul du retour attendu pour l'état présent 
        # le calcul 
        
        returns = []
        discounted_sum=0
        for r in reward_history[::-1]: 
            discounted_sum = r + gamma*discounted_sum
            returns.insert(0, discounted_sum)
        returns = np.array(returns)
        returns = (returns - np.mean(returns))/(np.std(returns)+eps)
        returns = returns.tolist()
        
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            adv = ret - value 
            actor_losses.append(-log_prob * adv)
            critic_losses.append(huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret,0))  )
            
        loss_value = sum(actor_losses) + sum(critic_losses)
        # print(loss_value)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        action_probs_history.clear()
        critic_value_history.clear()
        reward_history.clear()
        
        episode_count += 1
        if episode_count%10==0:
            print("Running reward {} at episode {}".format(running_reward, episode_count))
        
        if running_reward > 195: # Condition to consider the task solved 
            print("Solved at episode {}!".format(episode_count))
            break

***
Sauvegarde du modèle

In [ ]:
model.save('A2C')

***
A partir de l'environnement sauvegardé, l'environnement est joué pour 10000 épisodes. On réutilise donc les deux réseaux de neurones définis dans l'entraînement et l'action choisie est toujours issue de la distribution de probabilité de l'acteur. 

JOUER l'environnement avec l'entrainement effectué

In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keyboard

seed = 42
gamma = 0.99  # Discount factor for past rewards
max_steps_per_episode = 10000
env = gym.make("CartPole-v0")  # Create the environment
env.seed(seed)

In [ ]:
reward_history=[]
num_actions = 2

model = keras.models.load_model("A2C")
model.summary()

# while True:
state = env.reset()
    # episode_reward=0
with tf.GradientTape() as tape: # A retravailler
    for timestep in range(1, max_steps_per_episode):
        env.render() # Si on veux le rendu
        state = tf.convert_to_tensor(state)
        state = tf.expand_dims(state, 0)
            
            # pour l'état présent, renvoie les sorties des deux 2N Actor et Critic
            # stocke l'avantage pour des calculs ultérieurs hors de la boucle
        action_probs, critic_value = model(state)
            # critic_value_history.append(critic_value[0,0])
            # print(action_probs)
            # print(critic_value)
            # Policy : donne l'action selon les actions possibles et la distribution d'actions donnée par l'actor
            # puis stocke cette action dans l'historique
        action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            # action_probs_history.append(tf.math.log(action_probs[0,action]))
            
            # Joue l'environnement pour l'état présent et l'action précédemment choisie
            # puis stocke la récompense dans l'historique
            # Si done, alors le pendule est tombé et la simulation n'est plus intéressante.
        state, reward, done, _ = env.step(action)
        reward_history.append(reward)
            # episode_reward += reward
            
            # if done:
                # break 
                
            # print(timestep)]
    env.close()
    print("L'environnement Cartpole a terminé son exécution de {} itérations".format(max_steps_per_episode))